In [ ]:
import os
import srt
import asyncio
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from srt import compose
import nest_asyncio
import subprocess

nest_asyncio.apply()

# 定数
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MODEL_NAME = "gpt-4o-mini" 
TEMPERATURE = 0.1
MEMORY_SIZE = 20  
MAX_LINE_LENGTH = 25
MAX_LINES = 2

# LLMの初期化
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model_name=MODEL_NAME, temperature=TEMPERATURE)

# 翻訳用のプロンプトテンプレート（修正済み）
template = """あなたはプロの翻訳者であり、字幕翻訳の専門家です。以下の点を考慮して、自然で正確な日本語に翻訳してください。
* 口語的な表現を適切に使用する
* 文脈に合わせた適切な語彙を選択する
* 字幕のニュアンスや感情を正確に伝える
* 字幕のタイミングと長さを考慮し、簡潔かつ読みやすい表現にする

原文：
{text}
"""
prompt = ChatPromptTemplate.from_template(template)

# ConversationBufferWindowMemoryの初期化
memory = ConversationBufferWindowMemory(k=MEMORY_SIZE, input_key='text', memory_key='chat_history')

# 翻訳チェインの定義
chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

def get_srt_files():
    """現在のディレクトリ内のsrtファイルのリストを取得する。"""
    return [f for f in os.listdir('.') if f.endswith('.srt')]

def select_srt_file():
    """ユーザーにsrtファイルを選択させる。"""
    srt_files = get_srt_files()
    if not srt_files:
        print("No .srt files found in the current directory.")
        return None

    print("Available .srt files:")
    for i, file in enumerate(srt_files, 1):
        print(f"{i}. {file}")

    while True:
        try:
            choice = int(input("Enter the number of the file you want to use: "))
            if 1 <= choice <= len(srt_files):
                return srt_files[choice - 1]
            else:
                print("Invalid choice. Please try again.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def read_srt(input_srt_file):
    """SRTファイルを読み込み、字幕データのリストを返す。"""
    try:
        with open(input_srt_file, 'r', encoding='utf-8', errors='ignore') as file:
            srt_data = file.read()
        return list(srt.parse(srt_data))
    except Exception as e:
        print(f"Error reading SRT file: {e}")
        return []

async def translate_subtitles(subtitles):
    """字幕データを逐次的に翻訳し、メモリを更新する。"""
    total_subtitles = len(subtitles)
    for idx, subtitle in enumerate(subtitles):
        # 現在の字幕をmemoryに追加
        memory.save_context({"text": subtitle.content}, {"response": ""}) # responseは空文字列で初期化

        result = await chain.apredict(text=subtitle.content)
        subtitle.content = result.strip()

        # 翻訳結果をmemoryに追加
        memory.save_context({"text": subtitle.content}, {"response": result.strip()})

        progress = (idx + 1) / total_subtitles * 100
        print(f"Translation progress: {progress:.2f}%")
        print(f"Current Memory: {memory.buffer}") # デバッグ用にメモリ内容を出力
    return subtitles


def write_srt(output_srt_file, subtitles):
    """翻訳された字幕データをSRTファイルに書き込む。"""
    try:
        translated_srt_content = compose(subtitles)
        with open(output_srt_file, "w", encoding="utf-8") as f:
            f.write(translated_srt_content)
    except Exception as e:
        print(f"Error writing SRT file: {e}")

def add_line_breaks(text):
    """テキストに改行を追加する。"""
    import textwrap
    lines = textwrap.wrap(text, width=MAX_LINE_LENGTH)
    return '\n'.join(lines[:MAX_LINES])

def add_line_breaks_to_srt(srt_text):
    """SRT形式を保持して改行を追加する。"""
    subtitles = list(srt.parse(srt_text))
    for subtitle in subtitles:
        subtitle.content = add_line_breaks(subtitle.content)
    return compose(subtitles)

async def process_srt_file(input_srt_file, output_srt_file):
    """SRTファイルを読み込み、翻訳、改行を追加し、新しいSRTファイルに書き込む。"""
    subtitles = read_srt(input_srt_file)
    if not subtitles:
        print("No subtitles to translate.")
        return
    translated_subtitles = await translate_subtitles(subtitles)
    write_srt(output_srt_file, translated_subtitles)

    with open(output_srt_file, 'r', encoding='utf-8') as file:
        srt_text = file.read()

    result = add_line_breaks_to_srt(srt_text)

    with open("translated_file_result.srt", "w", encoding='utf-8') as file:
        file.write(result)

def create_video_with_subtitles(video_file, subtitle_file, output_video_file):
    """字幕付きのビデオを作成する。"""
    process = subprocess.Popen([
        'ffmpeg',
        '-hwaccel', 'auto',
        '-i', f"{video_file}",
        '-vf', f"subtitles={subtitle_file}:force_style='FontName=Helvetica,FontSize=11'",
        '-c:v', 'h264_amf',
        '-c:a', 'copy',
        '-progress', '-',  
        f'{output_video_file}'
    ], stderr=subprocess.PIPE, universal_newlines=True)

    while True:
        line = process.stderr.readline()
        if line == '' and process.poll() is not None:
            break
        if 'frame=' in line:
            print(line.strip())

    if process.returncode != 0:
        print(f"Error creating video: {process.returncode}")

# 実行
if __name__ == "__main__":

    input_srt = select_srt_file()
    if input_srt:
        output_srt = 'translated_' + input_srt
        video = input_srt.replace(".srt", ".mp4")
        output_video = video.replace(".mp4", "_jp.mp4")

        asyncio.run(process_srt_file(input_srt, output_srt))

        create_video_with_subtitles(video, "translated_file_result.srt", output_video)
    else:
        print("No file selected. Exiting.")